In [3]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.5
max_epoch = 3
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True


train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform = torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)

test_data = torchvision.datasets.MNIST(root = './mnist/',train = False)
test_x = Variable(torch.unsqueeze(test_data.test_data, dim = 1),volatile = True).type(torch.FloatTensor)[:500]/255.
test_y = test_data.test_labels[:500].numpy()

/root/anaconda3/lib/python3.11/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/tmp/ipykernel_7698/1855015138.py:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  test_x = Variable(torch.unsqueeze(test_data.test_data, dim = 1),volatile = True).type(torch.FloatTensor)[:500]/255.
/root/anaconda3/lib/python3.11/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [9]:
class myAlexNet(nn.Module):
    def __init__(self):
        super(myAlexNet, self).__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=96,
                kernel_size=5,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=96,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv_3 = nn.Sequential(
            nn.Conv2d(
                in_channels=256,
                out_channels=384,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv_4 = nn.Sequential(
            nn.Conv2d(
                in_channels=384,
                out_channels=384,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv_5 = nn.Sequential(
            nn.Conv2d(
                in_channels=384,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.fc_1 = nn.Sequential(
            nn.Linear(256 * 3 * 3, 4096),
            nn.ReLU(),
            nn.Dropout(keep_prob_rate),
        )

        self.fc_2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(keep_prob_rate),
        )

        self.fc_3 = nn.Sequential(
            nn.Linear(4096, 10),
        )
    def saveXasjpg(self,x):
        x = x.data.numpy()
        x = np.squeeze(x)
        x = (x - np.min(x)) / (np.max(x) - np.min(x))
        x = (x * 255).astype(np.uint8)
        for i in range(x.shape[0]):
            img = Image.fromarray(x[i])
            img.save(f'output_{i}.jpg')

    def forward(self, x):
        x = self.conv_1(x) # 1*28*28 -> 96*26*26
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 96*26*26 -> 96*13*13
        x = self.conv_2(x) # 96*13*13 -> 256*13*13
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 256*13*13 -> 256*6*6
        x = self.conv_3(x) # 256*6*6 -> 384*6*6
        x = self.conv_4(x) # 384*6*6 -> 384*6*6
        x = self.conv_5(x) # 384*6*6 -> 256*6*6
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 256*6*6 -> 256*3*3
        x = x.view(x.size(0), -1)
        x = self.fc_1(x)
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x
        
def test(alexnet):
    global prediction
    y_pre = alexnet(test_x)
    _,pre_index= torch.max(y_pre,1)
    pre_index= pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct  = np.sum(prediction == test_y)
    return correct / 500.0


def train(alexnet):
    optimizer = torch.optim.Adam(alexnet.parameters(), lr=learning_rate )
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x ,y= Variable(x_),Variable(y_)
            output = alexnet(x)  
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step != 0 and step % 20 ==0:
                print("=" * 10,step,"="*5,"="*5, "test accuracy is ",test(alexnet) ,"=" * 10 )

if __name__ == '__main__':
    alexnet = myAlexNet()
    train(alexnet)

========== 20 ===== ===== test accuracy is  0.172 ==========
========== 40 ===== ===== test accuracy is  0.502 ==========


KeyboardInterrupt: 